# Jeu de Morpion
Pour programmer un jeu, nous partons des principes suivants:
* des **variables d'état** décrivent la situation du jeu à un instant donné
* à chaque tour, les variables d'état changent
* les règles de jeu interdisent certains mouvements
* à chaque tour on vérifie si les critère pour gagner sont vérifiés
* une **matrice** représente le tableau de jeu
* des **entiers** (0 = vide, 1, 2, etc.) représentent les pièces

Avant de commencer, il faut importer le module `sense_hat` et `time` et eventuellent définir une rotation de l'écran de 180 degrés.

In [1]:
from sense_hat import SenseHat
#from sense_emu import SenseHat
from time import sleep, time

sense = SenseHat()
sense.set_rotation(180)

## Définition de l'état

Définissons l'état du jeu comme la matrice $3 \times 3$ avec des entiers ayant la signification:
* 0 case vide
* 1 case occupé par joueur 1
* 2 case occupé par joueur 2

Donc voici l'état initial du jeu

In [2]:
state = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
state

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

Supposons maintenant que joueur 1 place une pièce, et joueur 2 place sa pièce.

In [3]:
state = [[1, 2, 0], [0, 0, 0], [0, 0, 0]]
state

[[1, 2, 0], [0, 0, 0], [0, 0, 0]]

Définissons aussi une fonction `init()` pour initialiser l'état du jeu. La variable `state` étant une variable globale elle doit être déclarée avec le mot-clé `global` à l'intérieur d'une fonction.

In [4]:
def init():
    global state
    state = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

## Afficher l'état
Nous devons maintenant afficher cet état de jeu. Pour cette première version nous utilisons seulement $3 \times 3$ pixels. L'état du jeu est représenté par les 3 couleurs:
* dark green : case vide
* red : case occupé par joueur 1
* green : case occupé par jouer 2

In [7]:
dark_blue = (0, 0, 128)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
colors = (dark_blue, red, green)

def show(state):
    sense.clear()
    for y in range(3):
        for x in range(3):
            s = state[y][x]  # s is 0, 1, or 2
            sense.set_pixel(x, y, colors[s])          

In [6]:
show(state)

Supposons maintenant que joueur 1 place une pièce, et joueur 2 place sa pièce.

In [8]:
state = [[1, 2, 2], [1, 0, 0], [0, 0, 0]]
show(state)
state

[[1, 2, 2], [1, 0, 0], [0, 0, 0]]

Pour gagner, un jour doit occuper soit
* une des 3 lignes
* une des 3 colonnes
* une des 2 diagonales
Nous devons donc faire 8 comparaisons

Par exemple la première ligne (index 0) est

In [9]:
state[0]

[1, 2, 2]

De la première ligne (index 0) nous accédons le deuxième élément (index 1)

In [10]:
state[0][1]

2

## Critères pour gagner
Est-ce que les 3 éléments de la première ligne sont identiques ?

In [11]:
state[0][0] == state[0][1] == state[0][2]

False

Est-ce que les 3 éléments de la première ligne sont identiques ?

In [12]:
state[0][0] == state[0][1] == state[0][2]

False

Est-ce que les 3 éléments de la troisième ligne (index 2) sont identiques ? Oui, car les 3 cases sont vide (représenté par 0)

In [13]:
state[2][0] == state[2][1] == state[2][2]

True

Nous pouvons maintenant définir une fonction qui vérifie si le joueur $p$ (player) est dans une situation gagnante.

In [14]:
def winning(state, p):
    return  state[0][0] == state[0][1] == state[0][2] == p or \
            state[1][0] == state[1][1] == state[1][2] == p or \
            state[2][0] == state[2][1] == state[2][2] == p or \
            state[0][0] == state[1][0] == state[2][0] == p or \
            state[0][1] == state[1][1] == state[2][1] == p or \
            state[0][2] == state[1][2] == state[2][2] == p or \
            state[0][0] == state[1][1] == state[2][2] == p or \
            state[0][2] == state[1][1] == state[2][0] == p

Vérifions si joueur 1 ou joueur 2 est dans une situation gagnante.

In [15]:
winning(state, 1), winning(state, 2)

(False, False)

Si le jouer 1 ajoute encore une pièce, il peut gagner.

In [15]:
state = [[1, 2, 2], [1, 0, 0], [1, 0, 0]]
show(state)
state

[[1, 2, 2], [1, 0, 0], [1, 0, 0]]

In [17]:
winning(state, 1), winning(state, 2)

(True, False)

## Mouvement
Definissons maintenant la fonction `play(state, p`) qui définit le mouvement pour un joueur $p$. 
* La position initial du joueur est $(x, y) = (1, 1)$. 
* Le joueur peut se déplacer en direction $x$ et $y$ modulo 3.
* La position du joueur est affiché en bleu
* Appuyer sur le bouton central, change l'état et sort de la fonction

In [16]:
def play(state, p):
    (x, y) = (1, 1)
    dirs = {'up':(0, 1), 'down':(0, -1),
            'right':(1, 0), 'left':(-1, 0)}
    
    while True :
        for event in sense.stick.get_events():
            if event.action == 'pressed':
                if event.direction in dirs:
                    (dx, dy) = dirs[event.direction]
                    
                    x = (x + dx) % 3
                    y = (y + dy) % 3
                    show(state)
                    sense.set_pixel(x, y, blue)
                else:
                    state[y][x] = p
                    show(state)
                    return          

Voici le début d'un jeu avec un tour pour chaque joueur.

In [17]:
init()
show(state)
play(state, 1)
play(state, 2)

Maintenant nous pouvons combiner toutes les fonctions pour jouer un jeu entier:
* les joueurs 1 et 2 alternent avec l'expression `3 - player`
* quand un joueur gagne, le chiffre 1 ou 2 est affiché pendant 3 secondes
* ensuite le jeux est réinitialisé et continue avec le jouer suivant

In [20]:
init()
show(state)
win = False
player = 1

while True:
    play(state, player)
    if winning(state, player):
        sense.show_letter(str(player))
        sleep(3)
        init()
        show(state)
    player = 3 - player

KeyboardInterrupt: 